In [2]:
# === Import Libraries ===
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# === Color Palette ===
color_pallet = ["#c05095", "#8512a1", "#4d0d80", "#2f0e53", "#b63082"]

# === Load Data ===
data = pd.read_csv("/content/expanded_customer_support_dataset.csv")

# === Inspect Data ===
print(data.head())
print(data.info())

# === Check and Clean TotalCharges Column ===
# Find non-numeric values in TotalCharges
invalid_values = [val for val in data["TotalCharges"].unique() if not str(val).replace('.', '', 1).isdigit()]
print("Invalid TotalCharges values:", invalid_values)

# Replace space with NaN and convert to float
data["TotalCharges"] = data["TotalCharges"].replace(" ", np.nan).astype("float")

# Check missing values
print(data.isnull().sum())

# Fill missing TotalCharges where tenure is 0 with 0
data.loc[data["tenure"] == 0, "TotalCharges"] = 0

# Confirm cleanup
data.info()

# === Standardize Text Columns ===
data.replace({"No internet service": "No", "No phone service": "No"}, inplace=True)

# === Print Value Counts for Object Columns ===
for column in data.columns:
    if column == "customerID":
        continue
    if data[column].dtype == "O":
        print(f"Column: {column}")
        print(data[column].value_counts())
        print("-" * 50)

# === Basic Summary ===
print(data.describe(include="all").T)

# === Pie Chart: Churn Percentage ===
pie = go.Pie(
    labels=data["Churn"].value_counts().index,
    values=data["Churn"].value_counts().values,
    pull=[0.05, 0],
    rotation=10,
    marker_colors=color_pallet[2:]
)
fig = go.Figure(data=[pie])
fig.update_layout(title_text="Churn Percentage")
fig.show()

# === Bar Plot: Churn by Gender ===
gender_churn = data.groupby(["Churn", "gender"])["gender"].count().reset_index(name='count')
fig = px.bar(
    gender_churn,
    x='gender',
    y='count',
    color='Churn',
    barmode='group',
    text='count',
    title='Churn by Gender',
    color_discrete_sequence=color_pallet[1:]
)
fig.show()

# === Bar Plot: Churn by Senior Citizen ===
senior_churn = data.groupby(["Churn", "SeniorCitizen"])["gender"].count().reset_index(name="count")
fig = px.bar(
    senior_churn,
    x="SeniorCitizen",
    y="count",
    color="Churn",
    text="count",
    title='Churn by Senior Citizen',
    color_discrete_sequence=color_pallet[1:]
)
fig.show()

# === Box Plot: Monthly Charges by Senior ===
fig = px.box(data, x="SeniorCitizen", y="MonthlyCharges", points="all", color_discrete_sequence=color_pallet[1:], title="Seniors' Monthly Cost")
fig.show()

# === Box Plot: Monthly Charges by Contract and Senior ===
fig = px.box(
    data,
    x="Contract",
    y="MonthlyCharges",
    color="SeniorCitizen",
    color_discrete_sequence=color_pallet,
    title="Monthly Charges by Contract and Senior Status"
)
fig.show()

# === Box Plot: Monthly Charges by Payment Method and Contract ===
fig = px.box(
    data,
    x="PaymentMethod",
    y="MonthlyCharges",
    color="SeniorCitizen",
    facet_col="Contract",
    color_discrete_sequence=color_pallet,
    title="Monthly Charges by Payment Method and Contract"
)
fig.show()

# === Histogram: Total Charges by Senior ===
sns.histplot(data=data, x="TotalCharges", hue="SeniorCitizen", kde=True, palette=color_pallet, multiple="dodge")
plt.title("Total Charges by Senior Citizen")
plt.show()

# === Violin Plot: Tenure by Senior ===
fig = px.violin(data, y="tenure", color="SeniorCitizen", box=True, color_discrete_sequence=color_pallet, title="Tenure by Senior Citizen")
fig.show()

# === Histogram: Tenure by Churn ===
sns.histplot(data=data, x="tenure", hue="Churn", kde=True, palette=color_pallet, multiple="dodge")
plt.title("Tenure by Churn")
plt.show()

# === Boolean Column Mapping and KDE Plots ===
bool_data = data.copy()
bool_cols = []

for col in bool_data.columns:
    if bool_data[col].nunique() == 2:
        unique_vals = bool_data[col].dropna().unique()
        if set(unique_vals).issubset({"Yes", "No"}):
            bool_data[col] = bool_data[col].map({"Yes": True, "No": False})
            bool_cols.append(col)
        elif set(unique_vals).issubset({0, 1, True, False}):
            bool_cols.append(col)

# Plot histograms for Boolean columns
for col in bool_cols:
    plt.figure(figsize=(6, 4))
    sns.histplot(data=bool_data, x=col, hue="Churn", palette=color_pallet, multiple="dodge")
    plt.title(f'Churn Distribution for {col}')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.show()

# === Bar Plot: Churn by Internet Service ===
internet_churn = data.groupby(["InternetService", "Churn"])["Churn"].count().reset_index(name="count")
fig = px.bar(
    internet_churn,
    x='count',
    y='InternetService',
    color='Churn',
    barmode='group',
    text='count',
    title='Churn by Internet Service',
    color_discrete_sequence=color_pallet[1:]
)
fig.show()

# === Bar Plot: Monthly Charges Mean by Internet Service ===
internet_month_avg = data.groupby("InternetService")["MonthlyCharges"].mean().reset_index(name="MonthlyCharges Mean")
fig = px.bar(
    internet_month_avg,
    x='MonthlyCharges Mean',
    y='InternetService',
    color='InternetService',
    barmode='group',
    title='Monthly Charges Mean by Internet Service',
    color_discrete_sequence=color_pallet
)
fig.show()

                               Query          Intent  \
0        How do I reset my password?  reset_password   
1    What is the status of my order?    order_status   
2       Can I return a damaged item?     return_item   
3                Where is my refund?   refund_status   
4  How do I change my email address?    change_email   

                                            Response          Entity Language  \
0  To reset your password, click on 'Forgot Passw...        password  English   
1  Please provide your order ID to check the status.        order_id  English   
2  Yes, you can return damaged items within 30 da...  item_condition  English   
3    Refunds are processed within 5-7 business days.   refund_status  English   
4  You can change your email from the account set...           email  English   

  Channel Customer_ID   Timestamp  Response_Time_Seconds Issue_Resolved  \
0     Web    CUST1000  2024-01-01                     45            Yes   
1     Web    CUST1001  202

KeyError: 'TotalCharges'